# Latent Dirichlet Allocation LDA 

#### Wikifetcher
Raw Text von Wikipedia mittels Suchbegriffen
#### LDAbuilder
Ausführen der LDA mit der gegebenen Dokumentliste (Rohtext-Liste von Wikifetcher)

## Ausführung
### Konfiguration 
- Wir benötigen Zugriff auf Wikipedia für den Rohtext
- Natural Language Toolkit NLTK für die Tokenisierung und Stemming
- Stop_words, um nichtssagende Wörter zu entfernen
- Gensim für die Implementierung der Latent Dirichlet Allocation LDA

In [1]:
import wikipedia
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import re
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim import corpora, models

sentence_pat = re.compile(r'([A-Z][^\.!?]*[\.!?])', re.M)
tokenizer = RegexpTokenizer(r'\w+')

# Erzeuge englische stop words Liste
en_stop = get_stop_words('en')
# Erzeuge p_stemmer der Klasse PorterStemmer
p_stemmer = PorterStemmer()

doc_list = []
wikipedia.set_lang('en')

### Wikipedia Content
Mittels Suchbegriffen holen wir den Rohen Inhalt aus Wikipedia.
Danach wird der Inhalt in Sätze getrennt, welche zur Dokumentliste hinzugefügt werden.

In [2]:
def get_page(name):
    first_found = wikipedia.search(name)[0]
    try:
        return(wikipedia.page(first_found).content)
    except wikipedia.exceptions.DisambiguationError as e:
        return(wikipedia.page(e.options[0]).content)
    
search_terms = ['Stuttgart']
separator = '== References =='
for term in search_terms:
    full_content = get_page(term).split(separator, 1)[0]
    sentence_list = sentence_pat.findall(full_content)
    for sentence in sentence_list:
        doc_list.append(sentence)

    print(full_content[0:1500] + '...')

Stuttgart ( SHTUUT-gart; German: [ˈʃtʊtɡaʁt] ( listen); Swabian: Schduagert, pronounced [ˈʒ̊d̥ua̯ɡ̊ɛʕd̥]; names in other languages) is the capital and largest city of the German state of Baden-Württemberg.
Stuttgart is located on the Neckar river in a fertile valley locally known as the "Stuttgart Cauldron" an hour from the Swabian Jura and the Black Forest, and its urban area has a population of 609,219, making it the sixth largest city in Germany. 2.7 million people live in the city's administrative region and another 5.3 million people in its metropolitan area, making it the fourth largest metropolitan area in Germany.
The city and metropolitan area are consistently ranked among the top 20 European metropolitan areas by GDP; Mercer listed Stuttgart as 21st on its 2015 list of cities by quality of living, innovation agency 2thinknow ranked the city 24th globally out of 442 cities  and the Globalization and World Cities Research Network ranked the city as a Beta-status world city in t

### Vorverarbeitung
Der Text wird nun Tokenisiert, gestemt, nutzlose Wörter werden entfernt

In [3]:
num_topics = 4
num_words_per_topic = 15
texts = []

In [4]:
import pandas as pd

for doc in doc_list:
    raw = doc.lower()
    # Erzeuge tokens
    tokens = tokenizer.tokenize(raw)
    # Entferne unnütze Information
    stopped_tokens = [i for i in tokens if not i in en_stop]
    # Stemme tokens - Entfernung von Duplikaten und Transformation zu Grundform (Optional)
    # stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    texts.append(stopped_tokens)
output_preprocessed = pd.Series(texts)
print(output_preprocessed)

0      [stuttgart, shtuut, gart, german, ˈʃtʊtɡaʁt, l...
1      [stuttgart, located, neckar, river, fertile, v...
2                                              [germany]
3      [city, metropolitan, area, consistently, ranke...
4      [since, 6th, millennium, bc, stuttgart, area, ...
5      [roman, empire, conquered, area, 83, ad, built...
6      [stuttgart, s, roots, truly, laid, 10th, centu...
7      [overshadowed, nearby, cannstatt, town, grew, ...
8      [fortunes, stuttgart, turned, house, württembe...
9      [stuttgart, prospered, despite, setbacks, thir...
10     [however, 1952, city, bounced, back, became, m...
11     [stuttgart, also, transport, junction, possess...
12          [stuttgart, unusual, scheme, german, cities]
13     [spread, across, variety, hills, covered, vine...
14     [often, source, surprise, visitors, associate,...
15         [city, s, tourism, slogan, stuttgart, offers]
16     [current, plans, improve, transport, links, in...
17        [business, describes,

### Dictionary und Vektoren
In diesem Abschnitt wird nun der Bag-of-words Korpus erstellt. Die Vektoren werden später für das LDA-Modell benötigt

In [5]:
# Erzeuge ein dictionary
dictionary = corpora.Dictionary(texts)
# Konvertiere dictionary in Bag-of-Words
# corpus ist eine Liste von Vektoren - Jeder Dokument-Vektor ist eine Serie von Tupeln
corpus = [dictionary.doc2bow(text) for text in texts]

output_vectors = pd.Series(corpus)
print(output_vectors)

0      [(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1...
1      [(0, 2), (6, 1), (17, 1), (18, 1), (22, 1), (2...
2                                              [(41, 1)]
3      [(0, 1), (18, 4), (35, 1), (42, 2), (43, 1), (...
4      [(0, 1), (23, 1), (26, 1), (35, 2), (73, 1), (...
5      [(35, 1), (39, 1), (77, 1), (86, 1), (87, 1), ...
6      [(0, 1), (101, 1), (102, 1), (103, 1), (104, 1...
7      [(96, 1), (114, 1), (115, 1), (116, 1), (117, ...
8      [(0, 1), (16, 1), (21, 1), (106, 1), (122, 1),...
9      [(0, 1), (18, 1), (131, 1), (132, 1), (133, 1)...
10     [(18, 1), (98, 1), (143, 1), (144, 1), (145, 1...
11     [(0, 1), (17, 1), (40, 1), (41, 1), (154, 1), ...
12         [(0, 1), (3, 1), (56, 1), (169, 1), (170, 1)]
13     [(0, 1), (23, 1), (24, 1), (171, 1), (172, 1),...
14     [(18, 1), (141, 1), (182, 1), (183, 1), (184, ...
15     [(0, 1), (18, 1), (101, 1), (151, 1), (189, 1)...
16     [(0, 1), (18, 1), (155, 1), (189, 1), (191, 1)...
17              [(212, 2), (213

### LDA-Modell
Schließlich kann das LDA-Modell angewandt werden. Die Übergabeparameter dafür sind die Liste der Vektoren, die Anzahl der Themen, das Dictionary, sowie die Aktualisierungsrate.
In der Trainingsphase sollte eine höhere Aktualisierungsrate >= 20 gewählt werden.

In [6]:
# Wende LDA-Modell an
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=20)
lda = ldamodel.print_topics(num_topics=num_topics, num_words=num_words_per_topic)
    
for topic in lda:
    for entry in topic:
        print(entry)

0
0.025*"city" + 0.018*"stuttgart" + 0.012*"württemberg" + 0.009*"war" + 0.006*"s" + 0.005*"today" + 0.004*"000" + 0.004*"around" + 0.004*"first" + 0.004*"eberhard" + 0.004*"1" + 0.003*"army" + 0.003*"french" + 0.003*"number" + 0.003*"result"
1
0.024*"stuttgart" + 0.016*"s" + 0.015*"city" + 0.009*"germany" + 0.006*"german" + 0.005*"one" + 0.004*"world" + 0.004*"many" + 0.004*"area" + 0.004*"castle" + 0.003*"neckar" + 0.003*"population" + 0.003*"built" + 0.003*"history" + 0.003*"airport"
2
0.019*"stuttgart" + 0.009*"also" + 0.004*"university" + 0.004*"city" + 0.004*"germany" + 0.004*"cannstatt" + 0.004*"local" + 0.004*"old" + 0.004*"castle" + 0.004*"church" + 0.003*"württemberg" + 0.003*"since" + 0.003*"division" + 0.003*"time" + 0.003*"area"
3
0.042*"stuttgart" + 0.009*"württemberg" + 0.008*"city" + 0.008*"s" + 0.008*"state" + 0.007*"baden" + 0.007*"german" + 0.005*"also" + 0.004*"center" + 0.004*"world" + 0.004*"capital" + 0.004*"germany" + 0.004*"one" + 0.004*"cities" + 0.004*"univer